# Import Raw Data

In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [2]:
GESTURE_SIZE = 10

In [3]:
user = str(1)
raw_data = pd.read_csv('../raw-data/vol' + user + '.csv')

In [4]:
raw_data.head()

,Timestamp,X,Y,BTN_TOUCH,TOUCH_MAJOR,TOUCH_MINOR,TRACKING_ID,PRESSURE,FINGER
0,0.000000,350,1404,DOWN,11,-420,2526,38,0
1,0.032676,348,1406,HELD,11,-420,2526,38,0
2,0.042109,347,1408,HELD,11,-420,2526,38,0
3,0.049881,344,1411,HELD,11,-420,2526,38,0
4,0.058581,340,1416,HELD,11,-420,2526,38,0


# Prelim Data Cleaning

In [5]:
raw_data.sort_values(['FINGER','Timestamp'],inplace=True)  # Sort values by finger first, then timestamp

In [6]:
raw_data.dropna(inplace=True)

In [7]:
raw_data = pd.get_dummies(raw_data,columns=['BTN_TOUCH'],drop_first=True)  # drop first to prevent perfect predictor (overfitting)

# Feature Extraction

Feature Extraction Function

In [8]:
features = pd.DataFrame(columns=raw_data.columns)  # dataframe holding features for each gesture per row
features.insert(len(features.columns), "X_Speed", 0)
features.insert(len(features.columns), "X_Acceleration", 0)
features.insert(len(features.columns), "Y_Speed", 0)
features.insert(len(features.columns), "Y_Acceleration", 0)
features.insert(len(features.columns), "Speed", 0)
features.insert(len(features.columns), "Acceleration", 0)
features.insert(len(features.columns), "Jerk", 0)

In [9]:
def extract_features(gesture):
#     if type(gesture) != pd.DataFrame():
#         print("Error: Not DF")
    print(type(gesture))
    gesture.insert(len(gesture.columns), "X_Speed", 0)
    gesture.insert(len(gesture.columns), "X_Acceleration", 0)
    gesture.insert(len(gesture.columns), "Y_Speed", 0)
    gesture.insert(len(gesture.columns), "Y_Acceleration", 0)
    gesture.insert(len(gesture.columns), "Speed", 0)
    gesture.insert(len(gesture.columns), "Acceleration", 0)
    gesture.insert(len(gesture.columns), "Jerk", 0)
#     gesture.insert(len(gesture.columns), "Ang_V", 0)
#     gesture.insert(len(gesture.columns), "Path_Tangent", 0)
    
    gesture['X_Speed'] = (gesture.X - gesture.X.shift(1)) / (gesture.Timestamp - gesture.Timestamp.shift(1))
    gesture['Y_Speed'] = (gesture.Y - gesture.Y.shift(1)) / (gesture.Timestamp - gesture.Timestamp.shift(1))
    gesture['Speed'] = ((gesture.X_Speed ** 2) + (gesture.Y_Speed ** 2))**0.5
    gesture['X_Acceleration'] = (gesture.X_Speed - gesture.X_Speed.shift(1)) / (
                gesture.Timestamp - gesture.Timestamp.shift(1))
    gesture['Y_Acceleration'] = (gesture.Y_Speed - gesture.Y_Speed.shift(1)) / (
                gesture.Timestamp - gesture.Timestamp.shift(1))
    gesture['Acceleration'] = (gesture.Speed - gesture.Speed.shift(1)) / (
                gesture.Timestamp - gesture.Timestamp.shift(1))
    gesture['Jerk'] = (gesture.Acceleration - gesture.Acceleration.shift(1)) / (
                gesture.Timestamp - gesture.Timestamp.shift(1))
#     gesture['Path_Tangent'] = np.arctan2((gesture.Y - gesture.Y.shift(1)), (gesture.X - gesture.X.shift(1)))
#     gesture['Ang_V'] = (gesture.Path_Tangent - gesture.Path_Tangent.shift(1)) / (
#                 gesture.Timestamp - gesture.Timestamp.shift(1))

    return gesture

In [10]:
current_gesture = pd.DataFrame(columns=raw_data.columns)  # dataframe holding all rows of raw data for a single gesture

In [11]:
current_gesture.tail()

,Timestamp,X,Y,TOUCH_MAJOR,TOUCH_MINOR,TRACKING_ID,PRESSURE,FINGER,BTN_TOUCH_HELD,BTN_TOUCH_UP


In [12]:
len(current_gesture)

0

In [13]:
for row in raw_data.values:
    if len(current_gesture) >= GESTURE_SIZE:  # todo: include lifting finger as condition? and switching fingers
        features.loc[len(features.index)] = extract_features(current_gesture)  # append extracted features to features
#         print(features.loc[len(features.index)])
        current_gesture = pd.DataFrame(columns=current_gesture.columns)  # clear data but keep columns
        continue
    current_gesture.loc[len(current_gesture.index)] = row  # append row to current_gesture
#     print(row)


<class 'pandas.core.frame.DataFrame'>


ValueError: cannot set a row with mismatched columns

In [38]:
current_gesture.head()

,Timestamp,X,Y,TOUCH_MAJOR,TOUCH_MINOR,TRACKING_ID,PRESSURE,FINGER,BTN_TOUCH_HELD,BTN_TOUCH_UP


In [ ]:
features.head()